In [0]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
rag=np.load("/content/drive/My Drive/rag_features_small.npy")

In [0]:
cnn_features=pd.read_csv("/content/drive/My Drive/cnn_features_large.csv")

In [0]:
embedding=np.load("/content/drive/My Drive/graph_embedding.npy")

In [0]:
cnn_features.drop(['Unnamed: 0'],inplace=True,axis=1)

In [0]:
cnn_features=cnn_features.values

In [9]:
cnn_features.shape

(10015, 32)

In [0]:
total_features=np.hstack((rag,cnn_features,embedding))

In [11]:
total_features.shape

(10015, 6832)

In [0]:
def get_Y_orig(labels_loc):
    #output_list=list()
    Y_df = pd.read_csv(labels_loc)
    Y_df=Y_df.iloc[:,1:]
#     for index,row in Y_df.iterrows():
#         if row["MEL"]==1:
#             output_list.append(0)
#         elif row["NV"]==1:
#             output_list.append(1)
#         elif row["BCC"]==1:
#             output_list.append(2)
#         elif row["AKIEC"]==1:
#             output_list.append(3)
#         elif row["BKL"]==1:
#             output_list.append(4)
#         elif row["DF"]==1:
#             output_list.append(5)
#         elif row["VASC"]==1:
#             output_list.append(6)
    return Y_df

In [0]:
labels_loc = '/content/drive/My Drive/ISIC2018_target/ISIC2018_Task3_Training_GroundTruth.csv'
Y_orig = get_Y_orig(labels_loc)

In [15]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [16]:
model = Sequential()
model.add(Dense(12, input_dim=6832, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(7, activation='softmax'))

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
from sklearn.model_selection import train_test_split
train_X,val_X,train_Y,val_Y= train_test_split(total_features, Y_orig, test_size=0.2, shuffle= True)

In [19]:
model.fit(train_X, train_Y, epochs=100, batch_size=64,shuffle=True)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/100





8012/8012 [==============================] - 2s 208us/step - loss: 1.2245 - acc: 0.6534
Epoch 2/100
8012/8012 [==============================] - 1s 112us/step - loss: 0.6349 - acc: 0.8043
Epoch 3/100
8012/8012 [==============================] - 1s 83us/step - loss: 0.3808 - acc: 0.9073
Epoch 4/100
8012/8012 [==============================] - 1s 80us/step - loss: 0.2661 - acc: 0.9350
Epoch 5/100
8012/8012 [==============================] - 1s 83us/step - loss: 0.2037 - acc: 0.9465
Epoch 6/100
8012/8012 [==============================] - 1s 83us/step - loss: 0.1649 - acc: 0.9568
Epoch 7/100
8012/8012 [==============================] - 1s 85us/step - loss: 0.1370 - acc: 0.9623
Epoch 8/100
8012/8012 [==============================] - 1s 84us/step - loss: 0.1070 - acc: 0.9695
Epoch 9/100
8012/8012 [==============================] - 1s 83us/step - loss: 0.0819 - acc: 0.9797
Epoch

In [20]:
_, accuracy = model.evaluate(val_X, val_Y)
print('Accuracy: %.2f' % (accuracy*100))

2003/2003 [==============================] - 0s 70us/step
Accuracy: 95.26


In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 12)                81996     
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 63        
Total params: 82,163
Trainable params: 82,163
Non-trainable params: 0
_________________________________________________________________


In [0]:
predictions=model.predict_classes(val_X)

In [0]:
predictions=predictions.tolist()

In [0]:
output_list=list()
for index,row in val_Y.iterrows():
    if row["MEL"]==1:
        output_list.append(0)
    elif row["NV"]==1:
        output_list.append(1)
    elif row["BCC"]==1:
        output_list.append(2)
    elif row["AKIEC"]==1:
        output_list.append(3)
    elif row["BKL"]==1:
        output_list.append(4)
    elif row["DF"]==1:
        output_list.append(5)
    elif row["VASC"]==1:
        output_list.append(6)

In [0]:
from sklearn.metrics import classification_report

In [26]:
classification_report(output_list,predictions,output_dict=True)

{'0': {'f1-score': 0.8616780045351474,
  'precision': 0.867579908675799,
  'recall': 0.8558558558558559,
  'support': 222},
 '1': {'f1-score': 0.9705553484904957,
  'precision': 0.9644444444444444,
  'recall': 0.9767441860465116,
  'support': 1333},
 '2': {'f1-score': 0.9805825242718447,
  'precision': 0.9805825242718447,
  'recall': 0.9805825242718447,
  'support': 103},
 '3': {'f1-score': 0.9763779527559054,
  'precision': 0.9841269841269841,
  'recall': 0.96875,
  'support': 64},
 '4': {'f1-score': 0.9082774049217003,
  'precision': 0.9311926605504587,
  'recall': 0.8864628820960698,
  'support': 229},
 '5': {'f1-score': 0.9583333333333334,
  'precision': 1.0,
  'recall': 0.92,
  'support': 25},
 '6': {'f1-score': 1.0, 'precision': 1.0, 'recall': 1.0, 'support': 27},
 'accuracy': 0.9525711432850724,
 'macro avg': {'f1-score': 0.950829224044061,
  'precision': 0.9611323602956473,
  'recall': 0.9411993497528973,
  'support': 2003},
 'weighted avg': {'f1-score': 0.9523139513636616,
  '